# TASK1: load the toxic dataset and append all comments to one long text 

In [82]:
import pandas as pd

In [83]:
train_data = pd.read_csv('/home/ubuntu/Downloads/train.csv')
total_train_sent_list = list(train_data['comment_text'])

In [84]:
# long_text2 = train_data['comment_text'].str.cat()

In [85]:
long_text = ''
for line in total_train_sent_list:
    long_text += line
    long_text += ' '

# Task2: build vocabulary 

In [86]:
#we need that every characted will have an index 
long_text = long_text[:10000]
character_list = [character for character in long_text]
dict_character = dict(enumerate(set(character_list)))

In [90]:
reverese_dict = {value:key for key,value in dict_character.items()}

In [100]:
for index, ch in enumerate(sorted(set(character_list))):
    print (ch,index)


 0
  1
! 2
" 3
' 4
( 5
) 6
* 7
, 8
- 9
. 10
/ 11
0 12
1 13
2 14
3 15
4 16
5 17
7 18
8 19
9 20
: 21
; 22
? 23
A 24
B 25
C 26
D 27
E 28
F 29
G 30
H 31
I 32
J 33
K 34
L 35
M 36
N 37
O 38
P 39
R 40
S 41
T 42
U 43
V 44
W 45
Y 46
a 47
b 48
c 49
d 50
e 51
f 52
g 53
h 54
i 55
j 56
k 57
l 58
m 59
n 60
o 61
p 62
q 63
r 64
s 65
t 66
u 67
v 68
w 69
x 70
y 71
z 72
{ 73
| 74
} 75
~ 76


In [101]:
dict_character = {ch:index for index, ch in enumerate(sorted(set(character_list)))}

In [92]:
seq_len = 40
step = 3

X_list = []
Y_list = []
for start_index in list(range(0,len(long_text) - 100 ,step)):
    X_list.append(long_text[start_index:start_index + seq_len])
    Y_list.append(long_text[start_index + 1:start_index + seq_len + 1])

In [93]:
X_list[0]

'Nonsense?  kiss off, geek. what I said i'

In [68]:
Y_list[0]

'onsense?  kiss off, geek. what I said is'

# Task3: apply the dictionary 

In [94]:
X_list_tokenized = [dict_character[character] for character in line for line in X_list]

In [103]:
X_list_tokenized = []
for line in X_list:
    cur_line_values = []
    for ch in line:
        cur_line_values.append(dict_character[ch])
    X_list_tokenized.append(cur_line_values)

In [105]:
Y_list_tokenized = []
for line in Y_list:
    cur_line_values = []
    for ch in line:
        cur_line_values.append(dict_character[ch])
    Y_list_tokenized.append(cur_line_values)

In [108]:
X_list_tokenized[0]

[37,
 61,
 60,
 65,
 51,
 60,
 65,
 51,
 23,
 1,
 1,
 57,
 55,
 65,
 65,
 1,
 61,
 52,
 52,
 8,
 1,
 53,
 51,
 51,
 57,
 10,
 1,
 69,
 54,
 47,
 66,
 1,
 32,
 1,
 65,
 47,
 55,
 50,
 1,
 55]

In [109]:
Y_list_tokenized[0]

[61,
 60,
 65,
 51,
 60,
 65,
 51,
 23,
 1,
 1,
 57,
 55,
 65,
 65,
 1,
 61,
 52,
 52,
 8,
 1,
 53,
 51,
 51,
 57,
 10,
 1,
 69,
 54,
 47,
 66,
 1,
 32,
 1,
 65,
 47,
 55,
 50,
 1,
 55,
 65]